# RUL Transformer
This notebook creates a Transformer model for RUL prediction of the CMAPSS dataset.

In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

import DataUtils
import Masking

C:\Users\austi\miniconda3\envs\deep_learn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
batch_size = 64
trainloader, testloader = DataUtils.get_dataloaders(batch_size)

In [6]:
columns_excluded = [0, 1, 26]
columns_kept = [False, False, True, True, True,
               True, True, True, True, True,
               True, True, True, True, True,
               True, True, True, True, True,
               True, True, True, True, True,
               True, False]
print(columns_kept)

[False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False]


In [5]:
example = next(enumerate(trainloader))
ex_X = example[1][0]
print(ex_X.shape)

ex_y = example[1][1]
print(ex_y.shape)

torch.Size([64, 543, 27])
torch.Size([64, 543])


In [39]:
class RULTransformer(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.prediction_window = 1
        self.input_len = 543
        self.n_cols = 24
        self.embed_dim = 128
        
        self.input_projection = nn.Linear(self.input_len*self.n_cols, self.embed_dim)
        self.positional_embed = nn.Parameter(torch.randn(self.embed_dim))
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=self.embed_dim, nhead=4, activation="gelu")
        self.transformer_blocks = nn.TransformerEncoder(self.encoder_layer, num_layers=3)
        
        self.rul_head = nn.Sequential(nn.Linear(self.embed_dim, 2048),
                                           nn.ReLU(),
                                           nn.Linear(2048, self.prediction_window))
    
    def forward(self, x):
        pad_amount = self.input_len - x.size(1)
        x = torch.nn.functional.pad(x, (0, 0, 0, pad_amount))
        
        x = x.flatten(1)
        z = self.input_projection(x)
        z = z + self.positional_embed
        z = self.transformer_blocks(z)
        z = self.rul_head(z)
        
        return z.squeeze(1)

In [31]:
rul_tran = RULTransformer()

In [32]:
rul_tran(ex_X[:,:,columns_kept].float())

tensor([[-0.1573],
        [-0.1729],
        [-0.0386],
        [-0.0921],
        [-0.0726],
        [-0.1178],
        [-0.1323],
        [-0.1697],
        [ 0.0348],
        [-0.1476],
        [-0.1591],
        [-0.1775],
        [-0.0781],
        [-0.1042],
        [-0.2162],
        [-0.0993],
        [ 0.0056],
        [-0.1057],
        [-0.2110],
        [-0.2424],
        [-0.1530],
        [-0.1447],
        [-0.1584],
        [-0.0410],
        [-0.1164],
        [-0.0917],
        [-0.1578],
        [-0.1260],
        [-0.0889],
        [-0.1200],
        [-0.0743],
        [-0.0810],
        [-0.0902],
        [-0.1040],
        [-0.1477],
        [-0.0842],
        [-0.0747],
        [-0.1815],
        [-0.1360],
        [-0.1316],
        [-0.2062],
        [-0.1635],
        [-0.1210],
        [-0.1794],
        [-0.1585],
        [-0.1300],
        [-0.0380],
        [-0.1937],
        [-0.1327],
        [-0.1816],
        [-0.1397],
        [-0.1028],
        [-0.

In [41]:
objective = nn.MSELoss()

rul_tran = RULTransformer()

lr = 1e-4
n_epochs = 1000
optim = torch.optim.Adam(rul_tran.parameters(), lr=lr)
losses = []

for n in range(n_epochs):
    counter = 0
    for i, (X, y) in enumerate(tqdm(trainloader)):
        index = np.random.randint(0,543)
        optim.zero_grad()
        yhat = rul_tran(X[:,0:index,columns_kept].float())
#         print(y.shape)
        loss = objective(yhat, y[:,index].float())
        loss.backward()
        losses.append(loss.item())
        optim.step()
        counter += 1
        
    print("Epoch:", n+1, "Loss:",np.mean(losses[-counter:][0]))

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:05<00:00,  2.36it/s]


Epoch: 1 Loss: 25815.2578125


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:05<00:00,  2.33it/s]


Epoch: 2 Loss: 708.6223754882812


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:05<00:00,  2.38it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:05<00:00,  2.34it/s]


Epoch: 163 Loss: 378.2793884277344


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:05<00:00,  2.25it/s]


Epoch: 164 Loss: 4905.869140625


  8%|██████▉                                                                            | 1/12 [00:00<00:05,  1.96it/s]